In [ ]:
#| include: false
%load_ext autoreload
%autoreload 2

`gingado` seeks to facilitate the use of machine learning in economic and finance use cases, while promoting good practices. This package aims to be suitable for beginners and advanced users alike. Use cases may range from simple data retrievals to experimentation with machine learning algorithms to more complex model pipelines used in production.

## Overview

`gingado` is a free, open source library built different functionalities:

* **data augmentation**, to add more data from official sources, improving the machine models being trained by the user;

* **automatic benchmark model**, to enable the user to assess their models against a reasonably well-performant model; 

* *(new!)* **relevant datasets**, both real and simulamed, to allow for easier model development and comparison;

* **support for model documentation**, to embed documentation and ethical considerations in the model development phase; and

* **utils**, including tools to allow for lagging variables in a straightforward way.

Each of these functionalities builds on top of the previous one. They can be used on a stand-alone basis, together, or even as part of a larger pipeline from data input to model training to documentation!

:::{.callout-tip}

New functionalities are planned over time, so consider checking frequently on `gingado` for the latest toolsets.

:::

## Design principles

The choices made during development of `gingado` derive from the following principles, in no particular order:

* **flexibility**: users can use `gingado` out of the box or build custom processes on top of it;

* **compatibility**: `gingado` works well with other widely used libraries in machine learning, such as `scikit-learn` and `pandas`; and

* **responsibility**: `gingado` facilitates and promotes model documentation, including ethical considerations, as part of the machine learning development workflow.

## Acknowledgements

`gingado`'s API is inspired on the following libraries:

* `scikit-learn` ([API description](https://arxiv.org/abs/1309.0238))

* `keras` (website [here](https://keras.io/about/) and also, [this essay](https://medium.com/s/story/notes-to-myself-on-software-engineering-c890f16f4e4d))

* `fastai` ([description here](https://www.mdpi.com/2078-2489/11/2/108))

In addition, `gingado` is developed and maintained using [`nbdev`](https://nbdev.fast.ai).

## Presentations, talks, papers

The most current version of the paper describing `gingado` is [here](https://github.com/dkgaraujo/gingado_comms/blob/main/gingado.pdf). The paper and other material about `gingado` (ie, slide decks, papers) in [this dedicated repository](https://github.com/dkgaraujo/gingado_comms). Interested users are welcome to visit the repository and comment on the drafts or slide decks, preferably by opening an [issue](https://github.com/dkgaraujo/gingado_comms/issues). I also store in this repository suggestions I receive as issues, so users can see what others commented (anonymously unless requested) and comment along as well!

## Install

To install `gingado`, simply run the following code on the terminal:

`$ pip install gingado`

In [ ]:
#| echo: false
import nbdev; nbdev.nbdev_export()